## Look at the Data


In [1]:
# Import necesaary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import cvxpy as cp
import time

# Ensure reproducability
np.random.seed(0)

In [2]:
# Load the .mat file
data = loadmat("Data/mimo_detection.mat")

# Look at data
for  key, item in data.items():
    if type(item) != np.ndarray:
        print(f"{key}: {item}")
    else:
        print(f"{key}: shape {np.shape(item)}")
        print(f" Complex: {item.dtype.kind == 'c'}")
print("\n")

# Extract variables
Nrx = data['Nrx'][0][0]   # Number of receive antennas
print(f'Number of receive antennas: {Nrx}')
Ntx = data['Ntx'][0][0]  # Number of transmit antennas
print(f'Number of transmit antennas: {Ntx}')
SNR = data['snrdB'][0][0]
print(f'SNR: {SNR} dB')
Sc = data['sc']  # Transmitted symbols
Hc = data['Hc']  # Channel matrix
Yc = data['yc']  # Received symbols


__header__: b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Thu Dec  1 07:49:52 2016'
__version__: 1.0
__globals__: []
Hc: shape (40, 40)
 Complex: True
Nrx: shape (1, 1)
 Complex: False
Ntx: shape (1, 1)
 Complex: False
sc: shape (40, 1)
 Complex: True
snrdB: shape (1, 1)
 Complex: False
yc: shape (40, 1)
 Complex: True


Number of receive antennas: 40
Number of transmit antennas: 40
SNR: 20 dB


## Convert to appropriate form for optimization

In [3]:
# Make it a real-valued problem
H_real = np.real(Hc)
H_imag = np.imag(Hc)
H = np.block([[H_real, -H_imag],
              [H_imag, H_real]])
Y_real = np.real(Yc)
Y_imag = np.imag(Yc)
y = np.vstack((Y_real, Y_imag))
Sc_real = np.real(Sc)
Sc_imag = np.imag(Sc)
s_labels = np.vstack((Sc_real, Sc_imag))


# Make the matrices for the QP formulation
HtH = H.T @ H
Hty = H.T @ y
yty = float(y.T @ y)
A = np.block([
    [HtH,      -Hty],
    [-Hty.T,   np.array([[yty]])]
])


C:\Users\pablo\AppData\Local\Temp\ipykernel_28092\4287992554.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  yty = float(y.T @ y)


## Optimize using CVXPy Library

In [ ]:
# Parameters
N = 2 * Ntx + 1
eps = 1e-5
max_iters = 50000

# Define and solve the SDP relaxation
X = cp.Variable((N, N), PSD=True)
constraints = [cp.diag(X) == 1]
objective = cp.Minimize(cp.trace(A @ X))
prob = cp.Problem(objective, constraints)

# Solve
t0 = time.perf_counter()
prob.solve(solver=cp.CVXOPT, verbose=True)
t1 = time.perf_counter()

X_star = X.value

# Get results
print("status:", prob.status)
print("objective:", prob.value)
print(f"wall-clock solve time: {t1 - t0:.3f} s")

# Solver metadata 
# CVXPY stores solver-specific statistics here when available.
stats = prob.solver_stats
print("\n--- Solver stats (CVXPY) ---")
print("solver name:", stats.solver_name)
print("solve time reported by solver (s):", stats.solve_time)
print("setup time reported by solver (s):", stats.setup_time)
print("num iterations:", stats.num_iters)

# Check feasibility / quality on X_star 
print("\n--- Solution checks ---")

# Symmetry check
sym_err = np.linalg.norm(X_star - X_star.T, ord='fro') / max(1.0, np.linalg.norm(X_star, ord='fro'))
print(f"relative symmetry error: {sym_err:.2e}")

# Enforce symmetry for eigen-checks
X_sym = 0.5 * (X_star + X_star.T)

# Diagonal constraint satisfaction
diag_err = np.linalg.norm(np.diag(X_sym) - 1, ord=np.inf)
print(f"diag constraint max error ||diag(X)-1||_inf: {diag_err:.2e}")

# PSD check 
eigvals = np.linalg.eigvalsh(X_sym)
min_eig = eigvals.min()
print(f"min eigenvalue of X (symmetrized): {min_eig:.2e}")

# Rank-indicator / tightness check
trace_X = np.trace(X_sym)
top_frac = eigvals.max() / eigvals.sum()
print(f"trace(X): {trace_X:.6f} (should be ~{X_sym.shape[0]:d})")
print(f"largest-eigenvalue fraction lambda_max/sum(lambda): {top_frac:.4f}")

# Objective consistency: trace(A X) vs cvxpy value
obj_check = np.trace(A @ X_sym)
print(f"objective recomputed Tr(A X): {obj_check:.6e}")


                                     CVXPY                                     
                                     v1.6.5                                    
(CVXPY) Jan 08 10:27:08 PM: Your problem has 6561 variables, 81 constraints, and 0 parameters.
(CVXPY) Jan 08 10:27:08 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 08 10:27:08 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 08 10:27:08 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 08 10:27:08 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 08 10:27:08 PM: Compiling problem (target solver=CVXOPT).

## Extracting a Rank 1 Matrix from the result